# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [108]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [109]:
# 2) seu código aqui - Conte o número de valores missing por variável.

sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [110]:
# 3) seu código aqui - Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:

# criando uma df subset da tabela original com as colunas selecionadas

sinasc = sinasc.loc[:, ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [111]:
sinasc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     27028 non-null  int64  
 1   IDADEMAE    27028 non-null  int64  
 2   ESTCIVMAE   26711 non-null  float64
 3   ESCMAE      26716 non-null  object 
 4   QTDFILVIVO  25455 non-null  float64
 5   GESTACAO    25796 non-null  object 
 6   GRAVIDEZ    26949 non-null  object 
 7   CONSULTAS   27028 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.9+ MB


In [112]:
# 4) seu código aqui - Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.

sinasc.isna().sum() # contanto o número de missings

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [113]:
sinasc.index # contando o número de linhas = 27028

RangeIndex(start=0, stop=27028, step=1)

In [114]:
sinasc['APGAR5'].isna().sum() # identificado quantos são os dados ausentes em Apgar5

103

In [115]:
sinasc = sinasc[~sinasc['APGAR5'].isna()] # removendo todos os registros com Apgar5 não preenchidos
sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [121]:
sinasc['APGAR5'].isna().sum() # NAs da coluna eliminados

0

In [116]:
# 5) seu código aqui - observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9

sinasc.loc[:,['ESTCIVMAE','CONSULTAS']]

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [118]:
print(sinasc['ESTCIVMAE'].isna().sum()) # nessa coluna há 315 NAs restantes 
print(sinasc['CONSULTAS'].isna().sum()) # essa coluna não tem NAs

315
0


In [119]:
sinasc['ESTCIVMAE'] = sinasc['ESTCIVMAE'].fillna(9)

In [120]:
print(sinasc['ESTCIVMAE'].isna().sum()) # NAs tranformados em 9

0


In [99]:
sinasc['CONSULTAS'].isna().sum() # não é necessário rodar essa linha pois a variável não possui NAs

0

In [101]:
sinasc.isna().sum() # verificando os NAs restantes de todas as colunas

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [122]:
# 6) Seu código aqui - Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
sinasc['QTDFILVIVO'].isna().sum()

1566

In [123]:
sinasc['QTDFILVIVO'] = sinasc['QTDFILVIVO'].fillna(0) # substituindo NAs por 0

In [124]:
sinasc['QTDFILVIVO'].isna().sum() # Não há mais NAs na variável

0

In [129]:
# 7) seu código aqui - Das restantes, decida que valores te parecem mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.

sinasc.isna().sum() #verificando que 'ESCMAE', 'GESTACAO' e 'GRAVIDEZ' tem NAs

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

------------------------------------------ analisando 'GRAVIDEZ'

In [135]:
sinasc['GRAVIDEZ'].value_counts()

Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

In [137]:
sinasc['GRAVIDEZ'].isna().sum()

76

Decisão 1: Como quantidade de dados ausentes da variável 'GRAVIDEZ' me parecem poucos, acredito que não atrapalharia tanto na visualização ou mesmo produção de insights. No entanto, como a maioria esmagadora de mulheres tem gravidez Única (26356) substituirei por essa mesma categoria. o que não trará tanto impacto na df.

In [139]:
sinasc['GRAVIDEZ'] = sinasc['GRAVIDEZ'].fillna('Única') #substituindo os NAN por Única

In [140]:
sinasc['GRAVIDEZ'].isna().sum() # verificando se os NAN foram substituidos

0

------------------------------------------ analisando 'GESTACAO'

In [141]:
sinasc['GESTACAO'].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [142]:
sinasc['GESTACAO'].isna().sum()

1216

Decisão 2: 'GESTACAO' tem a maior quantidade de NAN e de acordo com os dados, eles parecem ser relevantes para identificar o apgar5 pois, parece-me, que qualquer alteração quanto a duração da gravidez pode ter impactos sobre a formação do bebê e suas características e habilidades desenvolvidas. Portanto, optei por preencher os dados também mas a partir do method = 'ffill', o q talvez dê mais homogeneidade aos dados

In [144]:
sinasc['GESTACAO'] = sinasc['GESTACAO'].fillna(method='ffill')

In [145]:
sinasc['GESTACAO'].isna().sum()

0

------------------------------------------ analisando 'ESCMAE'

In [146]:
sinasc['ESCMAE'].value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64

In [147]:
sinasc['ESCMAE'].isna().sum()

310

Decisão 3: COmo a escolaridade tem uma distribuição mais concentrada em torno das categorias que apontam um tempo maior, acima de 8 anos, há baixa probabilidade desta variável oferecer algum valor explicativo sobre o apgar5. Presumisse que quanto mais tempo de estudo, maior a probabilidade da mulher se informar sobre assuntos diversos e muito provável que se informe sobre sua gravidez também. Ademais, a probabillidade de ter ou não informação pode ter baixa influencia no desenvolvimento do bebê enquanto no útero. Logo, manter o dado como está, sem alterar ou remover os NANs, não me parece impactar tanto a visualização final.

In [154]:
# 8) seu código aqui - Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

# criando as categorias do APGAR5 na variável 'CAT_APGAR5'
sinasc.loc[sinasc['APGAR5'] <= 3, 'CAT_APGAR5'] = 'asfixia severa'
sinasc.loc[(sinasc['APGAR5'] >= 4 ) & (sinasc['APGAR5'] <= 5) , 'CAT_APGAR5'] = 'asfixia moderada'
sinasc.loc[(sinasc['APGAR5'] >= 6 ) & (sinasc['APGAR5'] <= 7) , 'CAT_APGAR5'] = 'asfixia leve'
sinasc.loc[sinasc['APGAR5'] >= 8 , 'CAT_APGAR5'] = 'normal'

Colei esse markdown aqui pra não ter que rolar a página de cima abaixo toda hora p lembrar as categorias

8 - O Apgar possui uma classificação indicando se o bebê passou por asfixia:

- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.

In [155]:
sinasc.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CAT_APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [157]:
sinasc['APGAR5'].value_counts().sort_index()

0.0        17
1.0        24
2.0        18
3.0        15
4.0        23
5.0        45
6.0        79
7.0       241
8.0      1994
9.0     16076
10.0     8393
Name: APGAR5, dtype: int64

In [161]:
freq_cat_apgar5 = (sinasc['APGAR5'].value_counts() / sinasc['APGAR5'].value_counts().sum()) * 100 # calculando frequencia
freq_cat_apgar5.sort_index()

0.0      0.063138
1.0      0.089136
2.0      0.066852
3.0      0.055710
4.0      0.085422
5.0      0.167131
6.0      0.293408
7.0      0.895079
8.0      7.405757
9.0     59.706592
10.0    31.171773
Name: APGAR5, dtype: float64

In [ ]:
# 9) seu código aqui - Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 

Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

In [177]:
sinasc.columns

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'CAT_APGAR5'],
      dtype='object')

In [175]:
col_names = sinasc.columns.str.lower() # passando para lower case
col_names

Index(['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao',
       'gravidez', 'consultas', 'apgar5', 'cat_apgar5'],
      dtype='object')

In [176]:
n_col_names = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao',
       'gravidez', 'consultas', 'apgar_5', 'cat_apgar5']
n_col_names # adicionando o underscore entre as palavras de forma manual. não sei se tem método p isso

['loc_nasc',
 'idade_mae',
 'est_civ_mae',
 'esc_mae',
 'qtd_fil_vivo',
 'gestacao',
 'gravidez',
 'consultas',
 'apgar_5',
 'cat_apgar5']

In [178]:
sinasc.columns = n_col_names # renomeando as colunas

In [179]:
sinasc.columns

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar_5', 'cat_apgar5'],
      dtype='object')

In [180]:
sinasc.head() # verificando se tudo está correto

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,cat_apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


FIM - que exercício longo